## Getting at Causality: An Illustrative Example of Regression with Dummy Variables

*Presented at [TogaData - Apr 9, 2017](https://www.meetup.com/Saratoga-Machine-Learning-and-Big-Data-Analytics/events/238983259/)*  
*by Chris Farr, PhD*

<!-- <img src="http://press.princeton.edu/images/k10363.gif" width="200" height="40"> -->

### Purpose
To demonstrate the use of **Regression with Dummy Variables** by reviewing an illustrative example from [Mastering 'Metrics: The Path from Cause to Effect](http://masteringmetrics.com) by Joshua D. Angrist and Steffen Pischke

### Agenda

1. The Question
1. Earnings Gap
1. The Idea
1. Earnings Gap When Controlling for Group Designation
1. Regression Model
1. The Design Matrix $X$
1. OLS Estimate
1. Wrap-Up


### 1. The Question

***What is the effect on earnings of attending a private versus public college?***

### 2. Earnings Gap

Let's look at an illustrative dataset (taken from [Mastering 'Metrics: The Path from Cause to Effect](http://masteringmetrics.com) by Joshua D. Angrist and Steffen Pischke, page 53) and compute the **Earnings Gap**.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Initial Data
df = pd.DataFrame([
        ['Private', 110000],
        ['Private', 100000],
        ['Public', 110000],
        ['Private', 60000],
        ['Public', 30000],
        ['Private', 115000],
        ['Private', 75000],
        ['Public', 90000],
        ['Public', 60000]
    ], columns=['Enroll_Dec', 'Earnings'])
df.index.name = 'Student'
print(df)

        Enroll_Dec  Earnings
Student                     
0          Private    110000
1          Private    100000
2           Public    110000
3          Private     60000
4           Public     30000
5          Private    115000
6          Private     75000
7           Public     90000
8           Public     60000


In [3]:
df.groupby('Enroll_Dec').mean()['Earnings']

Enroll_Dec
Private    92000
Public     72500
Name: Earnings, dtype: int64

Private students earned on average `$19,500` more than students who attended public colleges. Is this a valid comparison?

$$
\text{Raw Earnings Gap} = 92,000 - 72,500 = 19,500
$$

### 3. The Idea
The above points to a private-public earnings gap, yet we're not controlling for a host of other factors which could impact earnings like ability, SAT scores, gender, race, career ambition, family connections, family income and so on.

One approach is to group students based on the schools they both applied to and were admitted. Arguably, this controls for factors that impact earnings like ability -- and perhaps ambition -- and allows us to better isolate the effect on earnings of attending a private college. This doesn't control for all of the possible influences on earnings, but it's a start.

To do this, we augment the data above by grouping students into four groups:

Group | Student | Private <br/> Ivy | Private <br/> Leafy | Private <br/> Smart | Public <br/> All State | Public <br/> Tall State | Public <br> Alt State | Earnings $
:---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | ---:
A | 0 | | Reject | <span style="background-color: #FFFF00">**Enroll**</span> | | Admit | | 110,000
A | 1 | | Reject | <span style="background-color: #FFFF00">**Enroll**</span> | | Admit | | 100,000
A | 2 | | Reject | Admit | | <span style="background-color: #FFFF00">**Enroll**</span> | | 110,000
B | 3 | <span style="background-color: #FFFF00">**Enroll**</span> | | | Admit | | Admit | 60,000
B | 4 | Admit | | | Admit | | <span style="background-color: #FFFF00">**Enroll**</span> | 30,000
C | 5 | | <span style="background-color: #FFFF00">**Enroll**</span> | | | | | 115,000
C | 6 | | <span style="background-color: #FFFF00">**Enroll**</span> | | | | | 75,000
D | 7 | Reject | | | <span style="background-color: #FFFF00">**Enroll**</span> | Admit | | 90,000
D | 8 | Reject | | | Admit | <span style="background-color: #FFFF00">**Enroll**</span> | | 60,000

*Source: [Mastering 'Metrics: The Path from Cause to Effect](http://masteringmetrics.com) by Joshua D. Angrist and Steffen Pischke, page 53.*

In [4]:
# Let's put group assignment into our dataframe
df['Group'] = [ 'A', 'A', 'A', 'B', 'B', 'C', 'C', 'D', 'D']; df

,Enroll_Dec,Earnings,Group
Student,,,
0,Private,110000,A
1,Private,100000,A
2,Public,110000,A
3,Private,60000,B
4,Public,30000,B
5,Private,115000,C
6,Private,75000,C
7,Public,90000,D
8,Public,60000,D


### 4. Earnings Gap When Controlling for Group Designation

So, in a sense, we have constructed four *experiments* in order to calculate the private-public earnings gap. It would be nice to have more -- many more -- observations in each group, but, hey, this is an illustrative example.

Actually, we don't have four experiments, we have only two. Groups C and D are homogeneous and don't provide any information on the earnings gap: Group C has no students who went to public colleges, and Group D has no students who went to private colleges.

Now, let's look at average private and public earnings by group designation, ignoring the output from Groups C and D from the `groupby` call:

In [5]:
df.groupby(['Group','Enroll_Dec']).mean()['Earnings']

Group  Enroll_Dec
A      Private       105000
       Public        110000
B      Private        60000
       Public         30000
C      Private        95000
D      Public         75000
Name: Earnings, dtype: int64

We see that the private-public earnings differential, or gap, is the following for Groups A and B:

\begin{eqnarray}
\text{Group A Earnings Gap} &=& 105,000 - 110,000 &=& -5,000 \nonumber \\
\text{Group B Earnings Gap} &=& \:\:60,000 - \:\:30,000 &=& \:\:\:30,000 \nonumber \\
\end{eqnarray}

A simple average *across* Groups A and B yields a single estimate of the earnings gap:

$$
\frac{\big( -5,000 + 30,000 \big)}{2} = 12,500
$$

Alternatively, Angrist and Pischke suggest weighting the groups by number of students to compute the average:

$$
\Big( \frac{3}{5} \times -5,000 \Big) + \Big( \frac{2}{5} \times 30,000 \Big) = 9,000
$$

### 5. Regression Model

Now, with this intuition, let's do something more formal and run a regression with two dummy variables:

$$
Y_i = \alpha + \beta P_i + \gamma A_i + e_i
$$

Our parameters are defined as follows:

* $\alpha$ is the intercept
* $\beta$ is the causal effect on earnings of attending a private college
* $\gamma$ is the effect of being in Group A

The dependent and independent variables are defined as follows:

* $Y_i$ is the earnings for student $i$
* $P_i$ is our dummy variable with a value of **1** if student $i$ is enrolled in a private school and **0** otherwise
* $A_i$ is another dummy variable with a value of **1** if student $i$ is in Group A and a value of **0** otherwise

Recall Student 2's information:

In [6]:
# Student with index 2
df.iloc[2]

Enroll_Dec    Public
Earnings      110000
Group              A
Name: 2, dtype: object

Her feature vector is the following:

$$
x_2 = 
\begin{bmatrix}
    1 \\
    P_2 = 0 \\
    A_2 = 1 \\
\end{bmatrix}
$$

### 6. The Design Matrix $X$

To run the regression, we need to first arrange our data into a design matrix $X$, observations as rows and features as columns.

In [7]:
# Intercept
i = np.ones(df.shape[0]); i

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [8]:
# Private dummy
p = (df['Enroll_Dec'] == 'Private').values.astype(int); p

array([1, 1, 0, 1, 0, 1, 1, 0, 0])

In [9]:
# Group A dummy
a = (df['Group'] == 'A').values.astype(int); a

array([1, 1, 1, 0, 0, 0, 0, 0, 0])

In [10]:
# Groups C and D, Group B is reference group
c = (df['Group'] == 'C').values.astype(int)
d = (df['Group'] == 'D').values.astype(int)

In [11]:
# Design matrix X
# m - observations, n - features
# 1st column : constant term
# 2nd column : 1 - enrolled in private college, 0 - public college
# 3rd column : 1 - Group A, 0 - Group B
Xfull = np.stack((i, p, a, c, d), axis=-1); Xfull

array([[ 1.,  1.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  1.]])

In [12]:
X = Xfull[:5,:3]; X # for Groups A and B only

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  0.,  1.],
       [ 1.,  1.,  0.],
       [ 1.,  0.,  0.]])

Let's also define our earnings vector $y$:

In [13]:
# Earnings vector
yfull = np.array([110000, 100000, 110000, 60000, 30000,
               115000, 75000, 90000, 60000])
y = yfull[:5]; y # for Groups A and B only

array([110000, 100000, 110000,  60000,  30000])

### 7. OLS Estimate

We'll find $\hat{\theta}$ analytically by calculating

$$
\hat{\theta} = (X^T X)^{-1}X^Ty
$$

where $\hat{\theta}$ equals the $3 \times 1$ column vector of our parameter estimates

$$
\hat{\theta} = 
\begin{bmatrix}
    \hat{\alpha} \\
    \hat{\beta} \\
    \hat{\gamma} \\
\end{bmatrix}
$$

*See Prof. Strang's [Lecture 15](https://ocw.mit.edu/courses/mathematics/18-06-linear-algebra-spring-2010/video-lectures/lecture-15-projections-onto-subspaces/) for a refresher on solving least squares problems this way.*

In [14]:
# Ordinary Least Squares estimate, thetahat
thetahat = np.linalg.inv(X.T @ X) @ (X.T @ y); thetahat

array([ 40000.,  10000.,  60000.])

Thus, here are our estimates of the regression parameters:

$$
\hat{\theta} = 
\begin{bmatrix}
\hat{\alpha} = 40,000 \nonumber \\
\hat{\beta} = 10,000 \nonumber \\
\hat{\gamma} = 60,000 \nonumber \\
\end{bmatrix}
$$

In this example, we're more interested in the parameter estimates, namely the causal effect of private school education $\hat{\beta}$, but the below is what our model would deliver for predicted earnings for student $i$:

$$
\hat{Y}_i = 40,000 + 10,000 P_i + 60,000 A_i
$$

OLS effectively assigned the following weights to Group A and Group B:

$$
\Big( \frac{4}{7} \times -5,000 \Big) + \Big( \frac{3}{7} \times 30,000 \Big) = 10,000
$$

### 8. Wrap-Up

Let's recap our various estimates of the **Earnings Gap**:

\begin{eqnarray}
\text{Raw} &=& 19,500 \nonumber \\
\text{Simple Average} &=& 12,500 \nonumber \\
\text{Weighted Average} &=& \,\,\, 9,000 \nonumber \\
\text{OLS Estimate} &=& 10,000 \nonumber \\
\end{eqnarray}

### Appendix

#### a. Article which inspired the illustrative example

Stacy Berg Dale, Alan B. Krueger; **Estimating the Payoff to Attending a More Selective College: An Application of Selection on Observables and Unobservables.** Q J Econ 2002; 117 (4): 1491-1527. doi: 10.1162/003355302320935089

*Estimates of the effect of college selectivity on earnings may be biased because elite colleges admit students, in part, based on characteristics that are related to future earnings. We matched students who applied to, and were accepted by, similar colleges to try to eliminate this bias. Using the College and Beyond data set and National Longitudinal Survey of the High School Class of 1972, we find that students who attended more selective colleges earned about the same as students of seemingly comparable ability who attended less selective schools. Children from low-income families, however, earned more if they attended selective colleges.*

#### b. Using sklearn.linear_model.LinearRegression

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
# Implicit intercept
model = LinearRegression(fit_intercept=False)
model.fit(X,y)
model.coef_

array([ 40000.,  10000.,  60000.])

In [17]:
# Explicit intercept
model = LinearRegression(fit_intercept=True)
model.fit(X[:,1:3],y)
print(model.coef_)
print(model.intercept_)

[ 10000.  60000.]
40000.0


#### c. What happens when including Groups C and D in the Regression Model?

No change in estimate of $10,000 earnings differential as Groups C and D are homogeneous.

In [18]:
thetahat1 = np.linalg.inv(Xfull.T @ Xfull) @ (Xfull.T @ yfull); thetahat1

array([ 40000.,  10000.,  60000.,  45000.,  35000.])

#### d. Adjust for group effect on private school attendance then regress

Regress the private school dummy for individual $i$ who resides in group $j$ on the group dummies, where $\tilde{P}_{ij}$ is the residual term:

$$
P_{ij} = \pi_0 + \pi_1 A_{ij} + \pi_2 C_{ij} + \pi_3 D_{ij} + \tilde{P}_{ij}
$$

In [19]:
X2 = np.stack((i, a, c, d), axis=-1) # design matrix
thetahat2 = np.linalg.inv(X2.T @ X2) @ (X2.T @ p); thetahat2 # estimate of parameter vector

array([ 0.5       ,  0.16666667,  0.5       , -0.5       ])

The fitted values $\hat{\pi}_0 + \hat{\pi}_1 A_{ij} + \hat{\pi}_2 C_{ij} + \hat{\pi}_3 D_{ij}$ turns out to be the mean private school attendance rate in each group, $\bar{P}_j$.

In [20]:
# Fitted value, pbar
np.set_printoptions(precision=3, suppress=True) # w/o scientific notation
pbar = (X2 @ thetahat2); print(pbar) # fitted value

[ 0.667  0.667  0.667  0.5    0.5    1.     1.    -0.    -0.   ]


Thus, the residual can be written as follows:

$$
\tilde{P}_{ij} = P_{ij} - \bar{P_j}
$$

In [21]:
ptilde = p - pbar; ptilde # the residual

array([ 0.333,  0.333, -0.667,  0.5  , -0.5  ,  0.   ,  0.   ,  0.   ,  0.   ])

Now, let's finally run the following regression, where we'll see we obtain our original estimate of $\beta$

$$
Y_{ij} = \alpha + \beta \tilde{P}_{ij} + e_{ij}
$$

In [22]:
X3 = np.stack((i, ptilde), axis=-1) # design matrix
thetahat3 = np.linalg.inv(X3.T @ X3) @ (X3.T @ yfull); thetahat3[1] # estimate

10000.000000000016

, which can also be found by computing the following: $$
\beta = \frac{C(Y_{ij},\tilde{P}_{ij})}{V(\tilde{P}_{ij})}
$$

In [23]:
# Beta using covariance approach
np.cov(yfull,ptilde)[0][1]/np.cov(yfull,ptilde)[1][1]

10000.000000000015